In [2]:
# auto_label_yolo11.py
import argparse
import os
from pathlib import Path
from typing import List

import torch
from ultralytics import YOLO
from PIL import Image

def xyxy_to_yolo(box, img_w, img_h):
    x1, y1, x2, y2 = box
    w = max(x2 - x1, 0.0)
    h = max(y2 - y1, 0.0)
    cx = x1 + w / 2.0
    cy = y1 + h / 2.0
    # normalize
    return [
        (cx / img_w),
        (cy / img_h),
        (w / img_w),
        (h / img_h),
    ]

def list_images(folder: Path) -> List[Path]:
    exts = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp"}
    return sorted([p for p in folder.rglob("*") if p.suffix.lower() in exts])

def main():

    #------------------------------ Ayarlar ------------------------------

    img_dir = Path(r"C:\Users\524ha\Desktop\AYGAZ_DATAS\Datasets\yarımca_kamyon\Annotation\unlabeled\Yandan_0508\secondary_labeling\images")
    out_dir = Path(r"C:\Users\524ha\Desktop\AYGAZ_DATAS\Datasets\yarımca_kamyon\Annotation\unlabeled\Yandan_0508\secondary_labeling\labels")
    model_dir = Path(r"C:\Users\524ha\Desktop\AYGAZ_DATAS\Projects\ProjectDatas\models\kamyon\side\truck_side_0209.pt")

    args = {'conf': 0.2, 'iou': 0.2}
    #------------------------------ ------- ------------------------------

    if not img_dir.exists():
        raise SystemExit(f"Görsel klasörü bulunamadı: {img_dir}")
        

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Modeli yükle
    model = YOLO(model_dir)
    model.to(device)

    images = list_images(img_dir)
    if not images:
        raise SystemExit(f"Görsel bulunamadı: {img_dir}")

    print(f"{len(images)} görsel bulundu. Çıktılar: {out_dir.resolve()}")
    for img_path in images:
        # Görüntü boyutlarını al
        with Image.open(img_path) as im:
            img_w, img_h = im.size

        # Tahmin
        results = model.predict(
            source=str(img_path),
            conf=args['conf'],
            iou=args['iou'],
            device=device,
            verbose=False
        )

        # Her görüntü için tek sonuç nesnesi beklenir
        if len(results) == 0:
            print(f"Uyarı: Sonuç yok -> {img_path.name}")
            continue

        res = results[0]
        boxes = res.boxes  # Boxes object
        if boxes is None or len(boxes) == 0:
            # Boş txt de oluşturabilirsiniz isterseniz:
            open(out_dir / (img_path.stem + ".txt"), "w").close()
            print(f"Etiket yok (boş txt yazıldı): {img_path.name}")
            continue

        # YOLO txt satırlarını oluştur
        lines = []
        # boxes.xyxy, boxes.cls, boxes.conf mevcuttur
        xyxy = boxes.xyxy.cpu().numpy()
        cls = boxes.cls.cpu().numpy().astype(int)

        for i in range(len(xyxy)):
            c = cls[i]
            x1, y1, x2, y2 = xyxy[i]
            x, y, w, h = xyxy_to_yolo([float(x1), float(y1), float(x2), float(y2)], img_w, img_h)
            # format: class x_center y_center width height
            if c == 4:
                lines.insert(0, f"{c} {x:.6f} {y:.6f} {w:.6f} {h:.6f}")
            else:
                lines.append(f"{c} {x:.6f} {y:.6f} {w:.6f} {h:.6f}")

        # txt'yi yaz
        label_path = out_dir / (img_path.stem + ".txt")
        with open(label_path, "w", encoding="utf-8") as f:
            f.write("\n".join(lines))

        print(f"Yazıldı: {label_path.name}  (n={len(lines)})")

    print("Bitti ✅")

if __name__ == "__main__":
    main()


37 görsel bulundu. Çıktılar: C:\Users\524ha\Desktop\AYGAZ_DATAS\Datasets\yarımca_kamyon\Annotation\unlabeled\Yandan_0508\secondary_labeling\labels
Yazıldı: 0508_930.txt  (n=131)
Yazıldı: 0508_931.txt  (n=124)
Yazıldı: 0508_932.txt  (n=133)
Yazıldı: 0508_933.txt  (n=129)
Yazıldı: 0508_934.txt  (n=131)
Yazıldı: 0508_935.txt  (n=126)
Yazıldı: 0508_936.txt  (n=125)
Yazıldı: 0508_937.txt  (n=122)
Yazıldı: 0508_938.txt  (n=116)
Yazıldı: 0508_939.txt  (n=115)
Yazıldı: 0508_940.txt  (n=113)
Yazıldı: 0508_941.txt  (n=102)
Yazıldı: 0508_942.txt  (n=100)
Yazıldı: 0508_943.txt  (n=97)
Yazıldı: 0508_944.txt  (n=85)
Yazıldı: 0508_945.txt  (n=84)
Yazıldı: 0508_946.txt  (n=73)
Yazıldı: 0508_947.txt  (n=65)
Yazıldı: 0508_948.txt  (n=65)
Yazıldı: 0508_949.txt  (n=58)
Yazıldı: 0508_950.txt  (n=58)
Yazıldı: 0508_951.txt  (n=53)
Yazıldı: 0508_952.txt  (n=52)
Yazıldı: 0508_953.txt  (n=53)
Yazıldı: 0508_954.txt  (n=53)
Yazıldı: 0508_955.txt  (n=52)
Yazıldı: 0508_956.txt  (n=53)
Yazıldı: 0508_957.txt  (n=51)
